In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 19:27:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [16]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [19]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [22]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [26]:
from pyspark.sql import functions as F

In [28]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [29]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [30]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [33]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [40]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [35]:
df_trips_data.registerTempTable('trips_data')

In [38]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [45]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [49]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')

## Homework

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
!ls

03_test.ipynb		   download_data.sh
04_pyspark.ipynb	   fhv
05_taxi_schema.ipynb	   fhv_tripdata_2019-10.csv
06_spark_sql.ipynb	   fhvhv
06_spark_sql.py		   fhvhv_tripdata_2021-01.csv
06_spark_sql_big_query.py  fhvhv_tripdata_2021-01.parquet
07_groupby_join.ipynb	   head.csv
08_rdds.ipynb		   homework.ipynb
09_spark_gcs.ipynb	   taxi+_zone_lookup.csv
cloud.md		   zones


In [4]:
df_fhv = spark.read.parquet('fhv/2019/10')

In [7]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [6]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   null|                  null|
|              B01233|2019-10-08 00:09:01|2019-10-08 00:11:14|         264|          81|   null|                B01233|
|              B02688|2019-10-02 10:59:00|2019-10-02 11:34:46|         264|         107|   null|                B02688|
|              B01285|2019-10-03 21:56:10|2019-10-03 22:22:18|         264|         235|   null|                B01285|
|              B00706|2019-10-05 02:48:13|2019-10-05 02:56:36|         115|         245|   null|                B00706|
|              B02107|2019-10-05 08:04:3

In [8]:
df_fhv.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [9]:
df_fhv = df_fhv.withColumnRenamed('dropOff_datetime', 'dropoff_datetime')

In [15]:
df_fhv.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [11]:
df_fhv.registerTempTable('fhv_data')

/home/rcata/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
spark.sql("""
SELECT COUNT(*) FROM fhv_data WHERE pickup_datetime BETWEEN '2019-10-15 00:00:00' AND '2019-10-15 23:59:59'; 
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [18]:
from pyspark.sql.functions import col

In [20]:
df_fhv = df_fhv.withColumn("trip_duration_hours", (col("dropoff_datetime").cast("long") - col("pickup_datetime").cast("long")) / 3600)

# Find the maximum duration
max_trip_duration = df_fhv.agg({"trip_duration_hours": "max"}).collect()[0][0]

print("Length of the longest trip in the dataset:", max_trip_duration, "hours")

Length of the longest trip in the dataset: 631152.5 hours
